In [1]:
#VOTING

import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from nltk.classify import ClassifierI
from statistics import mode

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        return mode(votes)
    def confidence(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes)) #count how many occurences of most popular votes.
        conf = choice_votes / len(votes)
        return conf
        
        

documents=[(list(movie_reviews.words(fileid)),category)         #it contain all the words which has class label as positive or negative
           for category in movie_reviews.categories()
           for fileid in movie_reviews.fileids(category)]

#random.shuffle(documents)       #bcz postive and negative example are together...1000 postivie than 1000 negative

all_words=[]

for w in movie_reviews.words():         #append all the words in a list. so that we can classify.
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features= list(all_words.keys())[:3000]

def find_features(document):
    words=set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

#print(find_features(movie_reviews.words('neg/cv000_29416.txt')))

featuresets = [(find_features(rev),category) for (rev,category) in documents]
#print(featuresets[0:10])

#only positive testing set
training_set = featuresets[:1900]
testing_set = featuresets[1900:]

classifier= nltk.NaiveBayesClassifier.train(training_set)

print("original Naive Bayes Algo accuracy in %: ", nltk.classify.accuracy(classifier, testing_set)*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier Algo accuracy in %: ", nltk.classify.accuracy(MNB_classifier, testing_set)*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier Algo accuracy in %: ", nltk.classify.accuracy(BernoulliNB_classifier, testing_set)*100)



original Naive Bayes Algo accuracy in %:  77.0
Most Informative Features
                 idiotic = True              neg : pos    =     12.1 : 1.0
                  annual = True              pos : neg    =     10.7 : 1.0
               atrocious = True              neg : pos    =     10.5 : 1.0
                   sucks = True              neg : pos    =      9.5 : 1.0
                 frances = True              pos : neg    =      9.3 : 1.0
           unimaginative = True              neg : pos    =      7.5 : 1.0
                 cunning = True              pos : neg    =      7.0 : 1.0
             silverstone = True              neg : pos    =      6.9 : 1.0
                  sexist = True              neg : pos    =      6.9 : 1.0
                  regard = True              pos : neg    =      6.9 : 1.0
              schumacher = True              neg : pos    =      6.7 : 1.0
                  shoddy = True              neg : pos    =      6.3 : 1.0
                  suvari = 

In [2]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC


LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier Algo accuracy in %: ", nltk.classify.accuracy(LogisticRegression_classifier, testing_set)*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier Algo accuracy in %: ", nltk.classify.accuracy(SGDClassifier_classifier, testing_set)*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier Algo accuracy in %: ", nltk.classify.accuracy(SVC_classifier, testing_set)*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier Algo accuracy in %: ", nltk.classify.accuracy(LinearSVC_classifier, testing_set)*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier Algo accuracy in %: ", nltk.classify.accuracy(NuSVC_classifier, testing_set)*100)


#for voted classifier

voted_classifier = VoteClassifier(classifier,
                                 MNB_classifier,
                                 BernoulliNB_classifier,
                                 LogisticRegression_classifier,
                                 SGDClassifier_classifier,
                                 LinearSVC_classifier,
                                 NuSVC_classifier)

print("voted_classifier Algo accuracy in %: ", nltk.classify.accuracy(voted_classifier, testing_set)*100)
print("Classification :", voted_classifier.classify(testing_set[0][0]), " Confidence %: ", voted_classifier.confidence(testing_set[0][0])*100)


LogisticRegression_classifier Algo accuracy in %:  80.0
SGDClassifier_classifier Algo accuracy in %:  74.0
SVC_classifier Algo accuracy in %:  57.99999999999999
LinearSVC_classifier Algo accuracy in %:  81.0
NuSVC_classifier Algo accuracy in %:  80.0
voted_classifier Algo accuracy in %:  78.0
Classification : pos  Confidence %:  100.0
